## Connect Drive

In [1]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Mounted at /gdrive
/gdrive


In [2]:
cd 'My Drive/'

/gdrive/My Drive


# go to project folder

In [4]:
cd speech_separation

/gdrive/My Drive/speech_separation


### intsall packages needed

In [5]:
!pip install mtcnn

In [6]:
!pip install youtube-dl

In [7]:
!apt-get install libsox-fmt-all libsox-dev sox

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libsox-dev is already the newest version (14.4.2-3ubuntu0.18.04.1).
libsox-fmt-all is already the newest version (14.4.2-3ubuntu0.18.04.1).
sox is already the newest version (14.4.2-3ubuntu0.18.04.1).
0 upgraded, 0 newly installed, 0 to remove and 6 not upgraded.


In [8]:
import pandas as pd
import os

In [9]:
from data.lib import AVHandler as avh

In [10]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import sys
import datetime

In [11]:
from mtcnn.mtcnn import MTCNN
import cv2

In [12]:
import glob
import librosa
import numpy as np
import scipy.io.wavfile as wavfile

from model.lib import utils
import itertools
import time
import random
import math

In [13]:
import tensorflow as tf
from tensorflow.python.framework import tensor_util
from keras.models import load_model
from keras.models import Model
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [14]:
from keras import optimizers
from keras.models import Sequential, model_from_json
from keras import optimizers
from keras.layers import Input, Dense, Convolution2D, Bidirectional, UpSampling2D, UpSampling3D, concatenate
from keras.layers import Dropout, Flatten, BatchNormalization, ReLU, Reshape, Permute, Lambda, TimeDistributed
from keras.models import Model, load_model 
from keras.layers.recurrent import LSTM
from keras.initializers import he_normal, glorot_uniform
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, Callback
from keras.callbacks import TensorBoard
from model.lib.MyGenerator import AVGenerator

In [15]:
from model.lib.model_ops import ModelMGPU,latest_file
from keras import optimizers
from model.lib.model_loss import audio_discriminate_loss2 as audio_loss

### Dataset Path

In [16]:
cat_train = pd.read_csv('data/audio/catalog/avspeech_train.csv')
cat_train.columns = ['link','start_time','end_time','pos_x','pos_y']

In [17]:
cat_train.values.shape

(2621844, 5)

## Dataset Samples


In [18]:
# range for our dataset to be used
data_range = (1, 1000 ) # start index and ending index

# Step 2 - download audio

In [ ]:
# Before running, make sure avspeech_train.csv and avspeech_test.csv are in catalog.
# if not, see the requirement.txt
# download and preprocess the data from AVspeech dataset

def m_link(youtube_id):
    # return the youtube actual link
    link = 'https://www.youtube.com/watch?v='+youtube_id
    return link

def m_audio(loc,name,cat,start_idx,end_idx):
    # make concatenated audio following by the catalog from AVSpeech
    # loc       | the location for file to store
    # name      | name for the wav mix file
    # cat       | the catalog with audio link and time
    # start_idx | the starting index of the audio to download and concatenate
    # end_idx   | the ending index of the audio to download and concatenate

    for i in range(start_idx,end_idx):
        
        f_name = name+str(i)
        link = m_link(cat.loc[i,'link'])
        start_time = cat.loc[i,'start_time']
        end_time = start_time + 3.0
        avh.download(loc,f_name,link)
        avh.cut(loc,f_name,start_time,end_time)
        print('\r',i,'/',end_idx,end='')

# create 80000-90000 audios data from 290K
avh.mkdir('data/audio/audio_train')

start_time = time.time()
m_audio('data/audio/audio_train','audio_train',cat_train,data_range[0],data_range[1])
print( 'Time Taken', time.time() - start_time)

 404 / 1000

# Step 3 - Normalize audio data

In [ ]:
RANGE = data_range

if(not os.path.isdir('data/audio/norm_audio_train')):
    os.mkdir('data/audio/norm_audio_train')

start_time = time.time()

for num in range(RANGE[0],RANGE[1]+1):
    path = './data/audio/audio_train/trim_audio_train%s.wav'% num
    norm_path = './data/audio/norm_audio_train/trim_audio_train%s.wav'% num
    if (os.path.exists(path)):
        audio,_= librosa.load(path,sr=16000)
        max = np.max(np.abs(audio))
        norm_audio = np.divide(audio,max)
        wavfile.write(norm_path,16000,norm_audio)

print( 'Time Taken', time.time() - start_time)

# Step 4 - Downlaod videos

In [ ]:
def video_download(loc,cat,start_idx,end_idx):
    # Only download the video from the link
    # loc        | the location for downloaded file
    # v_name     | the name for the video file
    # cat        | the catalog with audio link and time
    # start_idx  | the starting index of the video to download
    # end_idx    | the ending index of the video to download

    for i in range(start_idx,end_idx):
        command = 'cd %s;' % loc
        f_name = str(i)
        link = avh.m_link(cat.loc[i, 'link'])
        start_time = cat.loc[i, 'start_time']
        end_time = start_time + 3.0
        start_time = datetime.timedelta(seconds=start_time)
        end_time = datetime.timedelta(seconds=end_time)
        command += 'ffmpeg -i $(youtube-dl -f ”mp4“ --get-url ' + link + ') ' + '-c:v h264 -c:a copy -ss %s -to %s %s.mp4' \
                % (start_time, end_time, f_name)
        #command += 'ffmpeg -i %s.mp4 -r 25 %s.mp4;' % (f_name,'clip_' + f_name) #convert fps to 25
        #command += 'rm %s.mp4' % f_name
        os.system(command)
        print('\r',i,'/',end_idx,end='')

def generate_frames(loc,start_idx,end_idx):
    # get frames for each video clip
    # loc        | the location of video clip
    # v_name     | v_name = 'clip_video_train'
    # start_idx  | the starting index of the training sample
    # end_idx    | the ending index of the training sample

    avh.mkdir('data/video/frames')
    for i in range(start_idx, end_idx):
        command = 'cd %s;' % loc
        f_name = str(i)
        command += 'ffmpeg -i %s.mp4 -y -f image2  -vframes 75 ../video/frames/%s-%%02d.jpg' % (f_name, f_name)
        os.system(command)


def download_video_frames(loc,cat,start_idx,end_idx,rm_video):
    # Download each video and convert to frames immediately, can choose to remove video file
    # loc        | the location for downloaded file
    # cat        | the catalog with audio link and time
    # start_idx  | the starting index of the video to download
    # end_idx    | the ending index of the video to download
    # rm_video   | boolean value for delete video and only keep the frames

    avh.mkdir('data/video/frames')
    for i in range(start_idx, end_idx + 1):
        command = 'cd %s;' % loc
        f_name = str(i)
        link = avh.m_link(cat.loc[i, 'link'])
        start_time = cat.loc[i, 'start_time']
        end_time = start_time + 3.0
        start_time = datetime.timedelta(seconds=start_time)
        end_time = datetime.timedelta(seconds=end_time)
        command += 'ffmpeg -i $(youtube-dl -f ”mp4“ --get-url ' + link + ') ' + '-c:v h264 -c:a copy -ss %s -to %s %s.mp4;' \
                   % (start_time, end_time, f_name)
        #ommand += 'ffmpeg -i %s.mp4 -r 25 %s.mp4;' % (f_name, 'clip_' + f_name)  # convert fps to 25
        #command += 'rm %s.mp4;' % f_name

        #converts to frames
        #command += 'ffmpeg -i %s.mp4 -y -f image2  -vframes 75 ../frames/%s-%%02d.jpg;' % (f_name, f_name)
        command += 'ffmpeg -i %s.mp4 -vf fps=25 ../video/frames/%s-%%02d.jpg;' % (f_name, f_name)
        #command += 'ffmpeg -i %s.mp4 ../frames/%sfr_%%02d.jpg;' % ('clip_' + f_name, f_name)

        if rm_video:
            command += 'rm %s.mp4' % f_name
        os.system(command)

avh.mkdir('data/video/video_train')

# download video , convert to images separately
#avh.video_download(loc='video_train',v_name='video_train',cat=cat_train,start_idx=2,end_idx=4)
#avh.generate_frames(loc='video_train',v_name='clip_video_train',start_idx=2,end_idx=4)
# download each video and convert to frames immediately

start_time = time.time()
download_video_frames(loc='data/video/video_train',cat=cat_train,start_idx=data_range[0],end_idx=data_range[1],rm_video=False)
print( 'Time Taken', time.time() - start_time)

# Step 5 - detect face

In [ ]:
frame_path = './data/video/frames/'
output_dir = './data/video/face_input'

if not os.path.isdir('./data/video/face_input'):
    os.mkdir('./data/video/face_input')

def bounding_box_check(faces,x,y):
    # check the center
    for face in faces:
        bounding_box = face['box']
        if(bounding_box[1]<0):
            bounding_box[1] = 0
        if(bounding_box[0]<0):
            bounding_box[0] = 0
        if(bounding_box[0]-50>x or bounding_box[0]+bounding_box[2]+50<x):
            print('change person from')
            print(bounding_box)
            print('to')
            continue
        if (bounding_box[1]-50 > y or bounding_box[1] + bounding_box[3]+50 < y):
            print('change person from')
            print(bounding_box)
            print('to')
            continue
        return bounding_box

def face_detect(file,detector,frame_path=frame_path,cat_train=cat_train):
    name = file.replace('.jpg', '').split('-')
    log = cat_train.iloc[int(name[0])]
    x = log['pos_x']
    y = log['pos_y']

    img = cv2.imread('%s%s'%(frame_path,file))
    x = img.shape[1] * x
    y = img.shape[0] * y
    faces = detector.detect_faces(img)
    # check if detected faces
    if(len(faces)==0):
        print('no face detect: '+file)
        return #no face
    bounding_box = bounding_box_check(faces,x,y)
    if(bounding_box == None):
        print('face is not related to given coord: '+file)
        return
    print(file," ",bounding_box)
    print(file," ",x, y)
    crop_img = img[bounding_box[1]:bounding_box[1] + bounding_box[3],bounding_box[0]:bounding_box[0]+bounding_box[2]]
    crop_img = cv2.resize(crop_img,(160,160))
    cv2.imwrite('%s/frame_'%output_dir + name[0] + '_' + name[1] + '.jpg', crop_img)
    #crop_img = cv2.cvtColor(crop_img, cv2.COLOR_BGR2RGB)
    #plt.imshow(crop_img)
    #plt.show()


start_time = time.time()
detector = MTCNN()
for i in range(data_range[0],data_range[1]+1):
    for j in range(1,76):
        file_name = "%d-%02d.jpg"%(i, j)
        if (not os.path.exists('%s%s' % (frame_path, file_name))):
            print('cannot find input: ' + '%s%s' % (frame_path, file_name))
            continue
        face_detect(file_name, detector)

print( 'Time Taken', time.time() - start_time)

### valid or not valid frame

In [ ]:
inspect_dir = 'data/video/face_input'
inspect_range = data_range
valid_frame_path = 'data/video/valid_frame.txt'

with open(valid_frame_path,'w+') as f:
  pass

def check_frame(idx,part,dir=inspect_dir):
    path = dir + "/frame_%d_%02d.jpg"%(idx,part)
#    print(path)
    if(not os.path.exists(path)): return False
    return True

start_time = time.time()
for i in range(inspect_range[0],inspect_range[1]+1):
    valid = True
#    print('processing frame %s'%i)
    for j in range(1,76):
        if(check_frame(i,j)==False):
            path = inspect_dir + "/frame_%d_*.jpg"% i
            for file in glob.glob(path):
                os.remove(file)
            valid = False
            print('frame %s is not valid'%i,j)
            break
    if valid:
        with open(valid_frame_path,'a') as f:
            frame_name = "frame_%d"%i
            f.write(frame_name+'\n')
            print(frame_name)

print( 'Time Taken', time.time() - start_time)

# Step 6 - Create audio database

In [ ]:
# Parameter
SAMPLE_RANGE = data_range # data usage to generate database
WAV_REPO_PATH = os.path.expanduser("./data/audio/norm_audio_train")
DATABASE_REPO_PATH = './data/audio/AV_model_database'
FRAME_LOG_PATH = './data/video/valid_frame.txt'
NUM_SPEAKER = 3
MAX_NUM_SAMPLE = 50000

# time measure decorator
def timit(func):
    def cal_time(*args,**kwargs):
        tic = time.time()
        result = func(*args,**kwargs)
        tac = time.time()
        print(func.__name__,'running time: ',(tac-tic),'ms')
        return result
    return cal_time

# create directory to store database
def init_dir(path = DATABASE_REPO_PATH ):
    if not os.path.isdir(path):
        os.mkdir(path)

    if not os.path.isdir('%s/mix'%path):
        os.mkdir('%s/mix'%path)

    if not os.path.isdir('%s/single'%path):
        os.mkdir('%s/single'%path)

    if not os.path.isdir('%s/crm'%path):
        os.mkdir('%s/crm'%path)

    if not os.path.isdir('%s/mix_wav'%path):
        os.mkdir('%s/mix_wav'%path)

@timit
def generate_path_list(sample_range=SAMPLE_RANGE,repo_path=WAV_REPO_PATH,frame_path=FRAME_LOG_PATH):
    '''

    :param sample_range:
    :param repo_path:
    :return: 2D array with idx and path (idx_wav,path_wav)
    '''
    audio_path_list = []
    frame_set = set()
    
    with open(frame_path,'r') as f:
        frames = f.readlines()
    
    for i in range(len(frames)):
        frame = frames[i].replace('\n','').replace('frame_','')
        frame_set.add(int(frame))
    
    for i in range(sample_range[0],(sample_range[1]+1)):
        print('\rchecking...%d'%int(frame),end='')
        path = repo_path + '/trim_audio_train%d.wav'%i
        if os.path.exists(path) and (i in frame_set):
            audio_path_list.append((i,path))
    print('\nlength of the path list: ',len(audio_path_list))
    return audio_path_list

# data generate function
def single_audio_to_npy(audio_path_list,database_repo=DATABASE_REPO_PATH,fix_sr=16000):
    for idx,path in audio_path_list:
        print('\rsingle npy generating... %d'%((idx/len(audio_path_list))*100),end='')
        data, _ = librosa.load(path, sr=fix_sr)
        data = utils.fast_stft(data)
        name = 'single-%05d'%idx
        with open('%s/single_TF.txt'%database_repo,'a') as f:
            f.write('%s.npy'%name)
            f.write('\n')
        np.save(('%s/single/%s.npy'%(database_repo,name)),data)
    print()


# split single TF data to different part in order to mix
def split_to_mix(audio_path_list,database_repo=DATABASE_REPO_PATH,partition=2):
    # return split_list : (part1,part2,...)
    # each part : (idx,path)
    length = len(audio_path_list)
    part_len = length // partition
    head = 0
    part_idx = 0
    split_list = []
    print('length = ',length,', parts = ',part_len)
    if part_len == 0:
      part_len = 1
    while((head+part_len)<=length):
        part = audio_path_list[head:(head+part_len)]
        split_list.append(part)
        with open('%s/single_TF_part%d.txt'%(database_repo,part_idx),'a') as f:
            for idx, _ in part:
                name = 'single-%05d' % idx
                f.write('%s.npy' % name)
                f.write('\n')
                print('\r',idx, end='')
        head += part_len
        part_idx += 1
    print('done')
    return split_list

# mix single TF data
def all_mix(split_list,database_repo=DATABASE_REPO_PATH,partition=2):
    print(len(split_list))
    assert len(split_list) == partition
    print('mixing data...')
    num_mix = 1
    num_mix_check = 0
    for part in split_list:
        num_mix *= len(part)
    print ('number of mix data; ',num_mix)

    part_len = len(split_list[-1])
    idx_list = [x for x in range(part_len)]
    combo_idx_list = itertools.product(idx_list,repeat=partition)
    for combo_idx in combo_idx_list:
        num_mix_check +=1
        single_mix(combo_idx,split_list,database_repo)
        print('\rnum of completed mixing audio : %d'%num_mix_check,end='')  
    print()
# mix several wav file and store TF domain data with npy
def single_mix(combo_idx,split_list,database_repo):
    assert len(combo_idx) == len(split_list)
    mix_rate = 1.0 / float(len(split_list))
    wav_list = []
    prefix = "mix"
    mid_name = ""

    for part_idx in range(len(split_list)):
        idx,path = split_list[part_idx][combo_idx[part_idx]]
        wav, _ = librosa.load(path, sr=16000)
        wav_list.append(wav)
        mid_name += '-%05d' % idx

    # mix wav file
    mix_wav = np.zeros_like(wav_list[0])
    for wav in wav_list:
        mix_wav += wav * mix_rate

    # save mix wav file
    wav_name = prefix+mid_name+'.wav'
    wavfile.write('%s/mix_wav/%s'%(database_repo,wav_name),16000,mix_wav)

    # transfer mix wav to TF domain
    F_mix = utils.fast_stft(mix_wav)
    name = prefix+mid_name+".npy"
    store_path = '%s/mix/%s'%(database_repo,name)

    # save mix as npy file
    np.save(store_path,F_mix)

    # save mix log
    with open('%s/mix_log.txt'%database_repo,'a') as f:
        f.write(name)
        f.write("\n")


def all_crm(mix_log_path,database_repo=DATABASE_REPO_PATH):
    with open(mix_log_path,'r') as f:
        mix_list = f.read().splitlines()

    for mix in mix_list:
        mix_path = '%s/mix/%s' % (database_repo, mix)
        mix = mix.replace(".npy","")
        mix = mix.replace("mix-","")
        idx_str_list = mix.split("-")
        single_crm(idx_str_list,mix_path,database_repo)

def single_crm(idx_str_list,mix_path,database_repo):
    F_mix = np.load(mix_path)
    mid_name = ""
    mix_name = "mix"
    dataset_line = ""

    for idx in idx_str_list:
        mid_name += "-%s"%idx
        mix_name += "-%s"%idx
    mix_name += '.npy'
    dataset_line += mix_name

    for idx in idx_str_list:
        single_name = 'single-%s.npy'%idx
        path = '%s/single/%s'%(database_repo,single_name)
        F_single = np.load(path)
        cRM = utils.fast_cRM(F_single,F_mix)

        last_name = '-%s'%idx
        cRM_name = 'crm' + mid_name + last_name + '.npy'

        # save crm to npy
        store_path = '%s/crm/%s'%(database_repo,cRM_name)
        np.save(store_path,cRM)

        # save crm information to log
        with open('%s/crm_log.txt'%database_repo, 'a') as f:
            f.write(cRM_name)
            f.write('\n')
        dataset_line += (" "+cRM_name)

    # write in database log
    with open('%s/dataset.txt'%database_repo,'a') as f:
        f.write(dataset_line)
        f.write('\n')


def train_test_split(dataset_log_path,data_range=[0,50000],test_ratio=0.1,shuffle=True,database_repo=DATABASE_REPO_PATH):
    with open(dataset_log_path,'r') as f:
        data_log = f.read().splitlines()

    if data_range[1]> len(data_log):
        data_range[1] = len(data_log)-1
    samples = data_log[data_range[0]:data_range[1]]
    if shuffle:
        random.shuffle(samples)

    length = len(samples)
    mid = int(math.floor(test_ratio*length))
    test = samples[:mid]
    train = samples[mid:]

    with open('%s/dataset_train.txt'%database_repo,'a') as f:
        for line in train:
            f.write(line)
            f.write('\n')

    with open('%s/dataset_val.txt' % database_repo, 'a') as f:
        for line in test:
            f.write(line)
            f.write('\n')


if __name__ == "__main__":
    init_dir()
    audio_path_list = generate_path_list()
    print('\nsingle audio to npy')
    single_audio_to_npy(audio_path_list)

    print('\nsplit to mix')
    start_time = time.time()
    split_list = split_to_mix(audio_path_list,partition=NUM_SPEAKER)
    print( time.time() - start_time )

    print('\nall mix')
    start_time = time.time()
    all_mix(split_list,partition=NUM_SPEAKER)
    print( time.time() - start_time )

    print('\nall crm')
    start_time = time.time()
    mix_log_path = '%s/mix_log.txt'%DATABASE_REPO_PATH
    all_crm(mix_log_path)
    print( time.time() - start_time )

    print('\ntrain test split')
    start_time = time.time()
    dataset_log_path ='%s/dataset.txt'%DATABASE_REPO_PATH
    train_test_split(dataset_log_path,data_range=[0,MAX_NUM_SAMPLE])
    print( time.time() - start_time )


# Step 7 - Generate log file for data generator

create AVdataset_train.txt

In [ ]:
with open('./data/audio/AV_model_database/dataset_train.txt', 'r') as t:
    lines = t.readlines()
    for line in lines:
        info = line.strip().split('.')
        num1 = info[0].strip().split('-')[1]
        num2 = info[0].strip().split('-')[2]

        newline = line.strip() + ' ' + num1 + '_face_emb.npy' + ' ' + num2 + '_face_emb.npy\n'
        with open('./data/AV_log/AVdataset_train.txt', 'a') as f:
            f.write(newline)

with open('./data/audio/AV_model_database/dataset_val.txt', 'r') as t:
    lines = t.readlines()
    for line in lines:
        info = line.strip().split('.')
        num1 = info[0].strip().split('-')[1]
        num2 = info[0].strip().split('-')[2]

        newline = line.strip() + ' ' + num1 + '_face_emb.npy' + ' ' + num2 + '_face_emb.npy\n'
        with open('./data/AV_log/AVdataset_val.txt', 'a') as f:
            f.write(newline)

# Step 8 - Generate Face Embedding

download facenet model

In [ ]:
start_time = time.time()
## paraeter
PB = 0
CKPT = 0
HDF5 = 1
MODEL_PATH = 'FaceNet_keras/facenet_keras.h5'
VALID_FRAME_LOG_PATH = './data/video/valid_frame.txt'
FACE_INPUT_PATH = './data/video/face_input/'

data = np.random.randint(256,size=(1,160,160,3),dtype='int32')

###############
#graph_path = 'FaceNet_new/20180402-114759.pb'
# utils.inspect_operation(graph_path,'ops.txt')
if PB:
    with tf.gfile.FastGFile(graph_path,'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        with tf.Session() as sess:
            #sess.graph.as_default()
            tf.import_graph_def(graph_def)
            print(sess.run('import/embeddings', feed_dict={'import/batch_size:0': data,'import/phase_train:0':False}))

if CKPT:
    saver = tf.train.import_meta_graph('FaceNet_new/model-20180402-114759.meta')
    with tf.Session() as sess:
        saver.restore(sess=sess,save_path='FaceNet_new/model-20180402-114759.ckpt-275')
        #print(sess.run('embeddings:0', feed_dict={'batch_size:0': data, 'phase_train:0': False}))

if HDF5:
    save_path = './data/video/face_emb/'
    if not os.path.exists(save_path):
        os.mkdir(save_path)

    model = load_model(MODEL_PATH)
    model.summary()
    avgPool_layer_model = Model(inputs=model.input,outputs=model.get_layer('AvgPool').output)
    # print(avgPool_layer_model.predict(data))

    lines = []
    with open(VALID_FRAME_LOG_PATH, 'r') as f:
        lines = f.readlines()

    for line in lines:
        embtmp = np.zeros((75, 1, 1792))
        headname = line.strip()
        tailname = ''
        for i in range(1, 76):
            if i < 10:
                tailname = '_0{}.jpg'.format(i)
            else:
                tailname = '_' + str(i) + '.jpg'
            picname = headname + tailname
            # print(picname)
            I = mpimg.imread(FACE_INPUT_PATH + picname)
            I_np = np.array(I)
            I_np = I_np[np.newaxis, :, :, :]
            # print(I_np.shape)
            # print(avgPool_layer_model.predict(I_np).shape)
            embtmp[i - 1, :] = avgPool_layer_model.predict(I_np)

        # print(embtmp.shape)
        people_index = int(line.strip().split('_')[1])
        npname = '{:05d}_face_emb.npy'.format(people_index)
        print(npname)

        np.save(save_path + npname, embtmp)
        with open('faceemb_dataset.txt', 'a') as d:
            d.write(npname + '\n')

print( 'Time Taken', time.time() - start_time)

In [ ]:
def AV_model(people_num=2):
    def UpSampling2DBilinear(size):
        return Lambda(lambda x: tf.compat.v1.image.resize_bilinear(x, size, align_corners=True))

    def sliced(x, index):
        return x[:, :, :, index]

    # --------------------------- AS start ---------------------------
    audio_input = Input(shape=(298, 257, 2))
    print('as_0:', audio_input.shape)
    as_conv1 = Convolution2D(96, kernel_size=(1, 7), strides=(1, 1), padding='same', dilation_rate=(1, 1), name='as_conv1')(audio_input)
    as_conv1 = BatchNormalization()(as_conv1)
    as_conv1 = ReLU()(as_conv1)
    print('as_1:', as_conv1.shape)

    as_conv2 = Convolution2D(96, kernel_size=(7, 1), strides=(1, 1), padding='same', dilation_rate=(1, 1), name='as_conv2')(as_conv1)
    as_conv2 = BatchNormalization()(as_conv2)
    as_conv2 = ReLU()(as_conv2)
    print('as_2:', as_conv2.shape)

    as_conv3 = Convolution2D(96, kernel_size=(5, 5), strides=(1, 1), padding='same', dilation_rate=(1, 1), name='as_conv3')(as_conv2)
    as_conv3 = BatchNormalization()(as_conv3)
    as_conv3 = ReLU()(as_conv3)
    print('as_3:', as_conv3.shape)

    as_conv4 = Convolution2D(96, kernel_size=(5, 5), strides=(1, 1), padding='same', dilation_rate=(2, 1), name='as_conv4')(as_conv3)
    as_conv4 = BatchNormalization()(as_conv4)
    as_conv4 = ReLU()(as_conv4)
    print('as_4:', as_conv4.shape)

    as_conv5 = Convolution2D(96, kernel_size=(5, 5), strides=(1, 1), padding='same', dilation_rate=(4, 1), name='as_conv5')(as_conv4)
    as_conv5 = BatchNormalization()(as_conv5)
    as_conv5 = ReLU()(as_conv5)
    print('as_5:', as_conv5.shape)

    as_conv6 = Convolution2D(96, kernel_size=(5, 5), strides=(1, 1), padding='same', dilation_rate=(8, 1), name='as_conv6')(as_conv5)
    as_conv6 = BatchNormalization()(as_conv6)
    as_conv6 = ReLU()(as_conv6)
    print('as_6:', as_conv6.shape)

    as_conv7 = Convolution2D(96, kernel_size=(5, 5), strides=(1, 1), padding='same', dilation_rate=(16, 1), name='as_conv7')(as_conv6)
    as_conv7 = BatchNormalization()(as_conv7)
    as_conv7 = ReLU()(as_conv7)
    print('as_7:', as_conv7.shape)

    as_conv8 = Convolution2D(96, kernel_size=(5, 5), strides=(1, 1), padding='same', dilation_rate=(32, 1), name='as_conv8')(as_conv7)
    as_conv8 = BatchNormalization()(as_conv8)
    as_conv8 = ReLU()(as_conv8)
    print('as_8:', as_conv8.shape)

    as_conv9 = Convolution2D(96, kernel_size=(5, 5), strides=(1, 1), padding='same', dilation_rate=(1, 1), name='as_conv9')(as_conv8)
    as_conv9 = BatchNormalization()(as_conv9)
    as_conv9 = ReLU()(as_conv9)
    print('as_9:', as_conv9.shape)

    as_conv10 = Convolution2D(96, kernel_size=(5, 5), strides=(1, 1), padding='same', dilation_rate=(2, 2), name='as_conv10')(as_conv9)
    as_conv10 = BatchNormalization()(as_conv10)
    as_conv10 = ReLU()(as_conv10)
    print('as_10:', as_conv10.shape)

    as_conv11 = Convolution2D(96, kernel_size=(5, 5), strides=(1, 1), padding='same', dilation_rate=(4, 4), name='as_conv11')(as_conv10)
    as_conv11 = BatchNormalization()(as_conv11)
    as_conv11 = ReLU()(as_conv11)
    print('as_11:', as_conv11.shape)

    as_conv12 = Convolution2D(96, kernel_size=(5, 5), strides=(1, 1), padding='same', dilation_rate=(8, 8), name='as_conv12')(as_conv11)
    as_conv12 = BatchNormalization()(as_conv12)
    as_conv12 = ReLU()(as_conv12)
    print('as_12:', as_conv12.shape)

    as_conv13 = Convolution2D(96, kernel_size=(5, 5), strides=(1, 1), padding='same', dilation_rate=(16, 16), name='as_conv13')(as_conv12)
    as_conv13 = BatchNormalization()(as_conv13)
    as_conv13 = ReLU()(as_conv13)
    print('as_13:', as_conv13.shape)

    as_conv14 = Convolution2D(96, kernel_size=(5, 5), strides=(1, 1), padding='same', dilation_rate=(32, 32), name='as_conv14')(as_conv13)
    as_conv14 = BatchNormalization()(as_conv14)
    as_conv14 = ReLU()(as_conv14)
    print('as_14:', as_conv14.shape)

    as_conv15 = Convolution2D(8, kernel_size=(1, 1), strides=(1, 1), padding='same', dilation_rate=(1, 1), name='as_conv15')(as_conv14)
    as_conv15 = BatchNormalization()(as_conv15)
    as_conv15 = ReLU()(as_conv15)
    print('as_15:', as_conv15.shape)

    AS_out = Reshape((298, 8 * 257))(as_conv15)
    print('AS_out:', AS_out.shape)
    # --------------------------- AS end ---------------------------

    # --------------------------- VS_model start ---------------------------
    VS_model = Sequential()
    VS_model.add(Convolution2D(256, kernel_size=(7, 1), strides=(1, 1), padding='same', dilation_rate=(1, 1), name='vs_conv1'))
    VS_model.add(BatchNormalization())
    VS_model.add(ReLU())
    VS_model.add(Convolution2D(256, kernel_size=(5, 1), strides=(1, 1), padding='same', dilation_rate=(1, 1), name='vs_conv2'))
    VS_model.add(BatchNormalization())
    VS_model.add(ReLU())
    VS_model.add(Convolution2D(256, kernel_size=(5, 1), strides=(1, 1), padding='same', dilation_rate=(2, 1), name='vs_conv3'))
    VS_model.add(BatchNormalization())
    VS_model.add(ReLU())
    VS_model.add(Convolution2D(256, kernel_size=(5, 1), strides=(1, 1), padding='same', dilation_rate=(4, 1), name='vs_conv4'))
    VS_model.add(BatchNormalization())
    VS_model.add(ReLU())
    VS_model.add(Convolution2D(256, kernel_size=(5, 1), strides=(1, 1), padding='same', dilation_rate=(8, 1), name='vs_conv5'))
    VS_model.add(BatchNormalization())
    VS_model.add(ReLU())
    VS_model.add(Convolution2D(256, kernel_size=(5, 1), strides=(1, 1), padding='same', dilation_rate=(16, 1), name='vs_conv6'))
    VS_model.add(BatchNormalization())
    VS_model.add(ReLU())
    VS_model.add(Reshape((75, 256, 1)))
    VS_model.add(UpSampling2DBilinear((298, 256)))
    VS_model.add(Reshape((298, 256)))
    # --------------------------- VS_model end ---------------------------

    video_input = Input(shape=(75, 1, 1792, people_num))
    AVfusion_list = [AS_out]
    for i in range(people_num):
        single_input = Lambda(sliced, arguments={'index': i})(video_input)
        VS_out = VS_model(single_input)
        AVfusion_list.append(VS_out)

    AVfusion = concatenate(AVfusion_list, axis=2)
    AVfusion = TimeDistributed(Flatten())(AVfusion)
    print('AVfusion:', AVfusion.shape)

    lstm = Bidirectional(LSTM(400, input_shape=(298, 8 * 257), return_sequences=True), merge_mode='sum')(AVfusion)
    print('lstm:', lstm.shape)

    fc1 = Dense(600, name="fc1", activation='relu', kernel_initializer=he_normal(seed=27))(lstm)
    print('fc1:', fc1.shape)
    fc2 = Dense(600, name="fc2", activation='relu', kernel_initializer=he_normal(seed=42))(fc1)
    print('fc2:', fc2.shape)
    fc3 = Dense(600, name="fc3", activation='relu', kernel_initializer=he_normal(seed=65))(fc2)
    print('fc3:', fc3.shape)

    complex_mask = Dense(257 * 2 * people_num, name="complex_mask", kernel_initializer=glorot_uniform(seed=87))(fc3)
    print('complex_mask:', complex_mask.shape)

    complex_mask_out = Reshape((298, 257, 2, people_num))(complex_mask)
    print('complex_mask_out:', complex_mask_out.shape)

    AV_model = Model(inputs=[audio_input, video_input], outputs=complex_mask_out)

    # # compile AV_model
    # AV_model.compile(optimizer='adam', loss='mse')

    return AV_model


if __name__ == '__main__':
    #############################################################
    RESTORE = False
    # If set true, continue training from last checkpoint
    # needed change 1:h5 file name, 2:epochs num, 3:initial_epoch

    # super parameters
    people_num = 2
    epochs = 50
    initial_epoch = 0
    batch_size = 1
    #############################################################

    # audio_input = np.random.rand(5, 298, 257, 2)        # 5 audio parts, (298, 257, 2) stft feature
    # audio_label = np.random.rand(5, 298, 257, 2, people_num)     # 5 audio parts, (298, 257, 2) stft feature, people num to be defined

    # ///////////////////////////////////////////////////////// #
    # create folder to save models
    path = './saved_models_AV'
    folder = os.path.exists(path)
    if not folder:
        os.makedirs(path)
        print('create folder to save models')
    filepath = path + "/AVmodel-" + str(people_num) + "p-{epoch:03d}-{val_loss:.10f}.h5"
    checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')


    # checkpoint2 = ModelCheckpoint(path + "/AOmodel-latest-" + str(people_num) + ".h5", monitor='val_loss', verbose=1, save_best_only=True, mode='min')
    # ///////////////////////////////////////////////////////// #

    #############################################################
    # automatically change lr
    def scheduler(epoch):
        ini_lr = 0.001
        lr = ini_lr
        if epoch >= 5:
            lr = ini_lr / 5
        if epoch >= 10:
            lr = ini_lr / 10
        return lr


    rlr = LearningRateScheduler(scheduler, verbose=1)
    #############################################################

    # ///////////////////////////////////////////////////////// #
    # read train and val file name
    # format: mix.npy single.npy single.npy
    trainfile = []
    valfile = []
#    with open('./data/audio/AV_model_database/dataset_train.txt', 'r') as t:
    with open('./data/AV_log/AVdataset_train.txt', 'r') as t:
        trainfile = t.readlines()

    validation = False
    if validation:
      with open('./data/AV_log/AVdataset_valid.txt', 'r') as v:
          valfile = v.readlines()

    # ///////////////////////////////////////////////////////// #

    # the training steps
    def latest_file(dir):
        lists = os.listdir(dir)
        lists.sort(key=lambda fn: os.path.getmtime(dir + fn))
        file_latest = os.path.join(dir, lists[-1])
        return file_latest


    if RESTORE:
        last_file = latest_file('./saved_models_AO/')
        AV_model = load_model(last_file)
        info = last_file.strip().split('-')
        initial_epoch = int(info[-2])
        # print(initial_epoch)
    else:
        AV_model = AV_model(people_num)
        adam = optimizers.Adam()
        AV_model.compile(optimizer=adam, loss='mse')

    train_generator = AVGenerator(trainfile, database_dir_path='./data/', batch_size=batch_size, shuffle=True)
    if validation:
      val_generator = AVGenerator(valfile, database_dir_path='./', batch_size=batch_size, shuffle=True)

    if validation:
      AV_model.fit_generator(generator=train_generator,
                            validation_data=val_generator,
                            epochs=epochs,
                            callbacks=[TensorBoard(log_dir='./log_AV'), checkpoint, rlr],
                            initial_epoch=initial_epoch
                            )
    else:
      AV_model.fit_generator(generator=train_generator,
                            epochs=epochs,
                            callbacks=[TensorBoard(log_dir='./log_AV'), checkpoint, rlr],
                            initial_epoch=initial_epoch
                            )



## train the model

In [ ]:
# create AV model
#############################################################
RESTORE = False
# If set true, continue training from last checkpoint
# needed change 1:h5 file name, 2:epochs num, 3:initial_epoch

# super parameters
people_num = 2
epochs = 100
initial_epoch = 0
batch_size = 2 # 4 to feed one 16G GPU
gamma_loss = 0.1
beta_loss = gamma_loss*2

# physical devices option to accelerate training process
workers = 1 # num of core
use_multiprocessing = False
NUM_GPU = 1

# PATH
path = './saved_AV_models' # model path
database_dir_path = './data/'
#############################################################

# create folder to save models
folder = os.path.exists(path)
if not folder:
    os.makedirs(path)
    print('create folder to save models')
filepath = path + "/AVmodel-" + str(people_num) + "p-{epoch:03d}-{val_loss:.5f}.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')


#############################################################
# automatically change lr
def scheduler(epoch):
    ini_lr = 0.00001
    lr = ini_lr
    if epoch >= 5:
        lr = ini_lr / 5
    if epoch >= 10:
        lr = ini_lr / 10
    return lr

rlr = LearningRateScheduler(scheduler, verbose=1)
#############################################################
# read train and val file name
# format: mix.npy single.npy single.npy
trainfile = []
valfile = []
with open((database_dir_path+'AV_log/AVdataset_train.txt'), 'r') as t:
    trainfile = t.readlines()
with open((database_dir_path+'AV_log/AVdataset_val.txt'), 'r') as v:
    valfile = v.readlines()
# ///////////////////////////////////////////////////////// #

# the training steps
if RESTORE:
    latest_file = latest_file(path+'/')
    AV_model = load_model(latest_file,custom_objects={"tf": tf})
    info = latest_file.strip().split('-')
    initial_epoch = int(info[-2])
else:
    AV_model = AV.AV_model(people_num)

train_generator = AVGenerator(trainfile,database_dir_path= database_dir_path, batch_size=batch_size, shuffle=True)
val_generator = AVGenerator(valfile,database_dir_path=database_dir_path, batch_size=batch_size, shuffle=True)

if NUM_GPU > 1:
    parallel_model = ModelMGPU(AV_model,NUM_GPU)
    adam = optimizers.Adam()
    loss = audio_loss(gamma=gamma_loss,beta=beta_loss,num_speaker=people_num)
    parallel_model.compile(loss=loss,optimizer=adam)
    print(AV_model.summary())
    parallel_model.fit_generator(generator=train_generator,
                           validation_data=val_generator,
                           epochs=epochs,
                           workers = workers,
                           use_multiprocessing= use_multiprocessing,
                           callbacks=[TensorBoard(log_dir='./log_AV'), checkpoint, rlr],
                           initial_epoch=initial_epoch
                           )
if NUM_GPU <= 1:
    adam = optimizers.Adam()
    loss = audio_loss(gamma=gamma_loss,beta=beta_loss, num_speaker=people_num)
    AV_model.compile(optimizer=adam, loss=loss)
    print(AV_model.summary())
    AV_model.fit_generator(generator=train_generator,
                           validation_data=val_generator,
                           epochs=epochs,
                           workers = workers,
                           use_multiprocessing= use_multiprocessing,
                           callbacks=[TensorBoard(log_dir='./log_AV'), checkpoint, rlr],
                           initial_epoch=initial_epoch
                           )

In [ ]:
## evaluate the model and generate the prediction
import sys
sys.path.append('../lib')
from keras.models import load_model
from model.lib.model_ops import ModelMGPU
import os
import scipy.io.wavfile as wavfile
import numpy as np
import tensorflow as tf
# super parameters
people_num = 2
NUM_GPU = 1

# PATH
model_path = './saved_models_AO_with_norm/AOmodel-2p-015-0.02258.h5'
dir_path = './pred/'
if not os.path.isdir(dir_path):
    os.mkdir(dir_path)
database_path = './data/audio/audio_database/mix/'
face_path = './data/video/face_emb/'

# load data
testfiles = []

with open('./data/AV_log/AVdataset_train.txt', 'r') as f:
    testfiles = f.readlines()

def parse_X_data(line,num_people=people_num,database_path=database_path,face_path=face_path):
    parts = line.split() # get each name of file for one testset
    mix_str = parts[0]
    name_list = mix_str.replace('.npy','')
    name_list = name_list.replace('mix-','',1)
    names = name_list.split('-')
    single_idxs = []
    for i in range(num_people):
        single_idxs.append(names[i])
    file_path = database_path + mix_str
    mix = np.load(file_path)
    face_embs = np.zeros((1,75,1,1792,num_people))
    for i in range(num_people):
        face_embs[1,:,:,:,i] = np.load(face_path+"%05d_face_emb.npy"%single_idxs[i])

    return mix,single_idxs,face_embs


# predict data
AV_model = load_model(model_path,custom_objects={"tf": tf})
if NUM_GPU > 1:
    parallel_model = ModelMGPU(AV_model,NUM_GPU)
    for line in testfiles:
        mix,single_idxs,face_embs = parse_X_data(line)
        mix_expand = np.expand_dims(mix, axis=0)
        cRMs = parallel_model.predict([mix_expand,face_embs])
        cRMs = cRMs[0]
        prefix = ""
        for idx in single_idxs:
            prefix += idx + "-"
        for i in range(len(cRMs)):
            cRM = cRMs[:,:,:,i]
            assert cRM.shape == (298,257,2)
            F = utils.fast_icRM(mix,cRM)
            T = utils.fast_istft(F,power=False)
            filename = dir_path+prefix+str(single_idxs[i])+'.wav'
            wavfile.write(filename,16000,T)


if NUM_GPU <= 1:
    for line in testfiles:
        mix, single_idxs, face_embs = parse_X_data(line)
        mix_expand = np.expand_dims(mix, axis=0)
        cRMs = AV_model.predict([mix_expand, face_embs])
        cRMs = cRMs[0]
        prefix = ""
        for idx in single_idxs:
            prefix += idx + "-"
        for i in range(people_num):
            cRM = cRMs[:,:,:,i]
            assert cRM.shape == (298,257,2)
            F = utils.fast_icRM(mix,cRM)
            T = utils.fast_istft(F,power=False)
            filename = dir_path+prefix+single_idxs[i]+'.wav'
            wavfile.write(filename,16000,T)
